In [2]:
import pandas as pd


In [3]:
# Path to train & test datasets
train_path= '../data/raw/training/UNSW_NB15_training-set.csv'
test_path= '../data/raw/testing/UNSW_NB15_testing-set.csv'

# Load train & test datasets
train_df= pd.read_csv(train_path)
test_df= pd.read_csv(test_path)

# Combine test$training for exploaration (but keep train/test info seprately)
df= pd.concat([train_df, test_df], ignore_index=True)

In [4]:
# Numbers of missing value per columns
print(df.isnull().sum())

id                   0
dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
attack_cat 

In [5]:
cat_cols= df.select_dtypes(include=['object']).columns.tolist()
for cols in cat_cols:
    print(df[cols].value_counts())

proto
tcp       123041
udp        92701
unas       15599
arp         3846
ospf        3271
           ...  
egp          131
netblt       131
igmp          48
icmp          15
rtp            1
Name: count, Length: 133, dtype: int64
service
-           141321
dns          68661
http         27011
smtp          6909
ftp-data      5391
ftp           4980
pop3          1528
ssh           1506
dhcp           120
snmp           109
ssl             86
irc             30
radius          21
Name: count, dtype: int64
state
FIN    117164
INT    116438
CON     20134
REQ      3833
RST        84
ECO        12
ACC         4
PAR         1
URN         1
no          1
CLO         1
Name: count, dtype: int64
attack_cat
Normal            93000
Generic           58871
Exploits          44525
Fuzzers           24246
DoS               16353
Reconnaissance    13987
Analysis           2677
Backdoor           2329
Shellcode          1511
Worms               174
Name: count, dtype: int64


In [6]:
# Seperate input features (x) from target label (y) for model training.

# Features
x= df.drop(columns=['label', 'attack_cat', 'id'])

# Target (binary)
y= df['label']

# Target (multi-class)
y_multi= df['attack_cat']

In [7]:
# Convert categorical variables to numerical variables
cat_cols= ['proto', 'service', 'state']
target_bin= 'label'
target_multi= 'attack_cat'
drop_col= ['id']

num_cols= [col for col in train_df if col not in cat_cols+[target_bin, target_multi]+drop_col]




In [9]:
# formating categories
for df_ in (train_df, test_df):
    df_['service']= df_['service'].replace('-', 'unknown').astype(str).str.strip().str.lower()
    for c in cat_cols: 
        df_[c] = df_[c].astype(str).str.strip().str.lower()

In [10]:
# collapse rare categories
min_count= 30
rare_proto= train_df['proto'].value_counts()[lambda x: x<min_count].index

train_df['proto']= train_df['proto'].where(train_df['proto'].isin(rare_proto), 'other')
test_df['proto']= test_df['proto'].where(test_df['proto'].isin(rare_proto), 'other')


In [12]:
# Build X and y
X_train= train_df.drop(columns=[target_bin, target_multi])
y_train= train_df[target_bin]

X_test= test_df.drop(columns=[target_bin, target_multi])
y_test= test_df[target_bin]



In [ ]:
# encode categories
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler    
